<a href="https://colab.research.google.com/github/imkaran23/Digit-recogniser/blob/master/DigitRecogniser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
!rm -rf ~/.kaggle
!mkdir ~/.kaggle
!cp "/content/drive/My Drive/kaggle.json" ~/.kaggle

In [0]:
!kaggle competitions download -c digit-recognizer

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 84.6MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 100MB/s]


In [0]:
!unzip test.csv.zip
!unzip train.csv.zip

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
#load the data
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [0]:
#get X_train,Y_train
X_train=train.drop(labels='label',axis=1)
Y_train=train['label']

#free space
del train

In [0]:
#check the data
X_train.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [0]:
test.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [0]:
# Normalization
X_train = X_train / 255.0
test = test / 255.0

In [0]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [0]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 10)

In [0]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state= 2)

In [0]:
# Set the CNN model: In -> [[Conv2D->relu]*3 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 14, 14, 64)      

In [0]:
# Defining optimizer
optimizer = RMSprop(lr=0.001, rho=0.9,epsilon=1e-08, decay=0.0)
# Compiling model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
# Reducing learning rate gradually
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
# Data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,            # set input mean to 0 over the dataset
        samplewise_center=False,             # set each sample mean to 0
        featurewise_std_normalization=False, # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,                 # apply ZCA whitening
        rotation_range=10,                   # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1,                    # Randomly zoom image 
        width_shift_range=0.1,               # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,              # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,               # randomly flip images
        vertical_flip=False)                 # randomly flip images

datagen.fit(X_train)

In [0]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=86),
                              epochs = 35, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/35
 - 11s - loss: 0.0393 - acc: 0.9882 - val_loss: 0.0406 - val_acc: 0.9886
Epoch 2/35
 - 11s - loss: 0.0377 - acc: 0.9889 - val_loss: 0.0224 - val_acc: 0.9936
Epoch 3/35
 - 11s - loss: 0.0358 - acc: 0.9891 - val_loss: 0.0284 - val_acc: 0.9917
Epoch 4/35
 - 11s - loss: 0.0337 - acc: 0.9902 - val_loss: 0.0238 - val_acc: 0.9921
Epoch 5/35
 - 11s - loss: 0.0331 - acc: 0.9902 - val_loss: 0.0415 - val_acc: 0.9902

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/35
 - 11s - loss: 0.0273 - acc: 0.9921 - val_loss: 0.0160 - val_acc: 0.9945
Epoch 7/35
 - 11s - loss: 0.0234 - acc: 0.9931 - val_loss: 0.0139 - val_acc: 0.9948
Epoch 8/35
 - 11s - loss: 0.0240 - acc: 0.9927 - val_loss: 0.0200 - val_acc: 0.9938
Epoch 9/35
 - 11s - loss: 0.0223 - acc: 0.9931 - val_loss: 0.0210 - val_acc: 0.9936
Epoch 10/35
 - 11s - loss: 0.0236 - acc: 0.9929 - val_loss: 0.0268 - val_acc: 0.9929

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.000250000011874362

In [0]:
# predict results
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [0]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("CNN_mnist_DataRecogniser.csv",index=False)